In [1]:
!pip install "tensorflow<2.11"

In [2]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        print("Using GPU:", gpus[0])
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found.")


Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [3]:
import os
import pandas as pd
PATH_DATA = "data"

images = []
labels = []

for subfolder in os.listdir(PATH_DATA):
    
    subfolder_path = os.path.join(PATH_DATA, subfolder)
    if not os.path.isdir(subfolder_path):
        continue
  
    for image_filename in os.listdir(subfolder_path):
        image_path = os.path.join(subfolder_path, image_filename)
        images.append(image_path)
    
        labels.append(subfolder)
 
df = pd.DataFrame({'image': images, 'label': labels})

In [4]:
from sklearn.model_selection import train_test_split

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

strat1 = df['label']
train_df, dummy_df = train_test_split(df,  train_size= 0.81, shuffle= True, random_state= 123, stratify= strat1)

strat2 = dummy_df['label']
valid_df, test_df = train_test_split(dummy_df,  train_size= 0.5, shuffle= True, random_state= 123, stratify= strat2)


tr_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
ts_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


train_ds = tr_gen.flow_from_dataframe(train_df, x_col='image', y_col='label', target_size=IMG_SIZE, class_mode='sparse', color_mode='rgb', shuffle=True, batch_size=BATCH_SIZE)

valid_ds = ts_gen.flow_from_dataframe(valid_df, x_col='image', y_col='label', target_size=IMG_SIZE, class_mode='sparse', color_mode='rgb', shuffle=True, batch_size=BATCH_SIZE)

test_ds = ts_gen.flow_from_dataframe(test_df, x_col='image', y_col='label', target_size=IMG_SIZE, class_mode='sparse', color_mode='rgb', shuffle=False, batch_size=BATCH_SIZE)

Found 3839 validated image filenames belonging to 4 classes.
Found 450 validated image filenames belonging to 4 classes.
Found 451 validated image filenames belonging to 4 classes.


In [5]:
# Normalization
def load_and_preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0
    return image

def preprocess_row(row):
    image = load_and_preprocess_image(row['image'])
    label = row['label']
    return image, label

# All Augmentation
def augmentation():
    augmentation_img = tf.keras.preprocessing.image.ImageDataGenerator( 
        rotation_range=30, 
        width_shift_range=0.1, 
        height_shift_range=0.1, 
        shear_range=0.2, 
        zoom_range=0.2, 
        horizontal_flip=True, 
        fill_mode='nearest' 
        ) 
    return augmentation_img

In [6]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint

model_dir = 'model'

class CustomCallback(Callback):
    def __init__(self, validation_data):
        super(CustomCallback, self).__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_loss = logs.get('val_loss', None)
        val_accuracy = logs.get('val_accuracy', None)

        # Contoh: Menampilkan metrik validasi
        print(f"\nEpoch {epoch + 1}:")
        print(f"  Validation Loss: {val_loss:.4f}")
        print(f"  Validation Accuracy: {val_accuracy:.4f}")
        
        # Contoh: Menyimpan metrik validasi ke file
        with open("training_log.txt", "a") as f:
            f.write(f"Epoch {epoch + 1}: Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}\n")

        # Tambahan: Bisa menghentikan pelatihan jika suatu kondisi terpenuhi
        if val_accuracy and val_accuracy > 0.95:
            print("Validation accuracy exceeded 95%. Stopping training!")
            self.model.stop_training = True

checkpoint = ModelCheckpoint(
    filepath=os.path.join(model_dir, 'best_model.keras'),  # Simpan model terbaik
    monitor='val_accuracy',        # Berdasarkan akurasi validasi
    save_best_only=True,           # Hanya menyimpan model terbaik
    mode='max',                    # Mencari nilai maksimum akurasi
    verbose=1
)

In [7]:
custom_callback = CustomCallback(validation_data=valid_ds)

# Gabungkan dengan callback lain
callbacks = [checkpoint, custom_callback]


In [8]:
# Optimalisasi
AUTOTUNE = tf.data.AUTOTUNE

# Membuat Dataset dari DataFrame
train_ds = tf.data.Dataset.from_tensor_slices(dict(train_df)).map(lambda x: preprocess_row(x), num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
valid_ds = tf.data.Dataset.from_tensor_slices(dict(valid_df)).map(lambda x: preprocess_row(x), num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = tf.data.Dataset.from_tensor_slices(dict(test_df)).map(lambda x: preprocess_row(x), num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [9]:
print(train_ds)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.string, name=None))>


In [10]:
# batch_size = 32
# img_shape = (224, 224, 3)

# train_gen = augmentation()
# test_gen = augmentation()

# train_gen = train_gen.flow_from_dataframe(train_df, x_col='image', y_col='label', target_size=(224, 224), class_mode='categorical', color_mode='rgb', shuffle=True, batch_size=batch_size)

# valid_gen = test_gen.flow_from_dataframe(valid_df, x_col='image', y_col='label', target_size=(224, 224), class_mode='categorical', color_mode='rgb', shuffle=True, batch_size=batch_size)

# test_gen = test_gen.flow_from_dataframe(test_df, x_col='image', y_col='label', target_size=(224, 224), class_mode='categorical', color_mode='rgb', shuffle=False, batch_size=batch_size)

In [11]:
# import matplotlib.pyplot as plt
# import numpy as np

# # Mengambil informasi kelas dari generator
# g_dict = train_gen.class_indices
# classes = list(g_dict.keys())

# # Mengambil batch pertama dari generator
# images, labels = next(train_gen)

# # Membuat figur
# plt.figure(figsize=(12, 12))

# # Looping untuk menampilkan gambar dalam grid 4x4
# for i in range(16):
#     plt.subplot(4, 4, i + 1)  # Grid 4x4
#     image = images[i] / 255  # Normalisasi gambar
#     plt.imshow(image)        # Menampilkan gambar

#     # Mendapatkan nama kelas berdasarkan label one-hot
#     index = np.argmax(labels[i])
#     class_name = classes[index]

#     # Menambahkan judul
#     plt.title(class_name, color='blue', fontsize=10)
#     plt.axis('off')  # Menyembunyikan sumbu

# # Menampilkan semua gambar
# plt.tight_layout()  # Memperbaiki tata letak subplot
# plt.show()


In [12]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization

def model_1():
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.InputLayer(input_shape=(224, 224, 3)))

    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)))

    model.add(Dense(4, activation='softmax'))

    return model

In [13]:
model = model_1()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 28, 28, 128)      0

In [14]:
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.utils import to_categorical

# # Membuat instance LabelEncoder
# label_encoder = LabelEncoder()

# # Melatih encoder pada label string dan mengonversinya menjadi label integer
# train_labels = label_encoder.fit_transform(train_labels)
# valid_labels = label_encoder.transform(valid_labels)
# test_labels = label_encoder.transform(test_labels)

# # Cek hasil konversi
# print(train_labels[:10])  # Memeriksa label pertama setelah konversi

# # One-hot encoding jika menggunakan categorical_crossentropy
# train_labels_one_hot = to_categorical(train_labels, num_classes=4)
# valid_labels_one_hot = to_categorical(valid_labels, num_classes=4)
# test_labels_one_hot = to_categorical(test_labels, num_classes=4)



In [15]:
model.compile(
    optimizer='adam',  # Adam optimizer adalah pilihan umum yang baik
    loss='sparse_categorical_crossentropy',  # Loss function untuk klasifikasi multi-class
    metrics=['accuracy'] 
)

history = model.fit(
    train_ds,                              # Dataset pelatihan
    validation_data=valid_ds,              # Dataset validasi
    epochs=50,                             # Maksimal epoch
    callbacks=callbacks,                   # Menggunakan callback
    verbose=1                              # Progress output
)

Epoch 1/50


UnimplementedError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/Cast' defined at (most recent call last):
    File "c:\Users\Lenovo\miniconda3\envs\tf\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Lenovo\miniconda3\envs\tf\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\Lenovo\miniconda3\envs\tf\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\Lenovo\miniconda3\envs\tf\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\Lenovo\miniconda3\envs\tf\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\Lenovo\miniconda3\envs\tf\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\Lenovo\miniconda3\envs